In [12]:
#PREPROCESS_hypergeometric_test_for_metabolomics    
#
#make data table that is ready for hypergeometric test for metabolomics
#seperately done for up, down

import pandas as pd
import numpy as np

In [13]:
def make_subpathway_dict(data_file):
    
    pathway_info_dict = {}
    chem_pathway_info_dict = {}
    
    data_df = pd.read_csv(chem_info_file, sep=',')
    r, c = data_df.shape
    chem_list = []
    
    for i in range(r):
        super_pathway = data_df["SUPER_PATHWAY"][i]
        sub_pathway = data_df["SUB_PATHWAY"][i]        
        chem_name = data_df["CHEMICAL_NAME"][i]
            
        #handle feature name differences
#         chem_name = chem_name.replace(',','_')
        try: 
            sub_pathway = sub_pathway.replace('_','')
            pathway = '[%s] %s' % (super_pathway, sub_pathway)
            if chem_name not in chem_list:
                chem_list.append(chem_name)
        except AttributeError: 
            sub_pathway = 'none'
            pathway = '[%s] %s' % (super_pathway, sub_pathway)

        try: pathway_info_dict[pathway].append(chem_name)
        except KeyError: pathway_info_dict[pathway] = [chem_name]

        chem_pathway_info_dict[chem_name] = pathway
    
    return pathway_info_dict, chem_pathway_info_dict, chem_list

def get_feature_list(data_file):
    
    feature_list = []
    open_data = open(data_file,'r')
    data_readlines = open_data.readlines()
    for i in range(len(data_readlines)):
        read = data_readlines[i]
        read = read.replace('\n','')
        feature_list.append(read)
        
    return feature_list

def make_hypergeometric_ready_matrix(feature_list, chem_pathway_info_dict, pathway_info_dict, chem_list, output_file):
    
    num_sig_feature = len(feature_list)
    pathway_dict = {}
    mentioned_pathway_list = []
    #summarize counts
    for feature in feature_list:
        pathway = chem_pathway_info_dict[feature]
        if pathway != "[nan] none":
            try: pathway_dict[pathway] += 1
            except KeyError: pathway_dict[pathway] = 1

            if pathway not in mentioned_pathway_list:
                mentioned_pathway_list.append(pathway)
    
    output_txt = open(output_file, 'w')
    output_txt.write('pathway\tnumber_of_metabolites_in_pathway(m)\tsignificant_metabolites_in_pathway(x)\ttotal_number_of_metabolites(n)\tdetected_significant_metabolites(k)\n')
    for pathway in mentioned_pathway_list:
        m = len(pathway_info_dict[pathway])
        k = pathway_dict[pathway]
        
        output_txt.write('%s\t%s\t%s\t%s\t%s\n' % (pathway, m, k, total_number_of_metabolites,num_sig_feature))


In [14]:
metabolomic_file_dir = '../../../../analysis/statistics/gse/metabolomics'
chem_info_file = '../../../../raw_data/RA_metabolon/MAYO-03-21ML+/RA_ACPA_metabolomics_chem_info.csv'

pathway_info_dict, chem_pathway_info_dict, chem_list = make_subpathway_dict(chem_info_file)

In [15]:
regulation_list = ["up","down"]
condition_list = ["cVSra","cVSneg","cVSpos","negVSpos"]
total_number_of_metabolites = 1061 #IMPORTANT: this number is "AFTER" prepreocessing the metabolomic data.

for condition in condition_list:
    for regulation in regulation_list:
        print (condition, regulation)
        data_file = '%s/metabolomics.%s.%s.tsv' % (metabolomic_file_dir, condition, regulation)
        output_file = '%s/metabolomics.%s.%s.hypergeometric.ready.tsv' % (metabolomic_file_dir, condition, regulation)
        feature_list = get_feature_list(data_file)

        make_hypergeometric_ready_matrix(feature_list, chem_pathway_info_dict, pathway_info_dict, total_number_of_metabolites, output_file)


cVSra up
cVSra down
cVSneg up
cVSneg down
cVSpos up
cVSpos down
negVSpos up
negVSpos down
